# 美团北京星巴克数据分析

网页：https://bj.meituan.com/s/%E6%98%9F%E5%B7%B4%E5%85%8B/

分析目标:
1. 所有门店按评分从高到低排序
2. 通过门店的商圈 （areaname）结合北京区域数据给每个门店添加新字段 area 表示其所属的北京区域（如朝阳区、西城区）
3. 按照北京区域分组所有门店
4. 统计各个区域的门店数量，找出拥有星巴克门店数量最多的三个区域
5. 统计各个区域星巴克门店的平均评分，找出平均评分最好的北京区域
6. 找出拥有最多星巴克门店且平均评分最高的商圈（areaname）

In [1]:
import csv
from pprint import pprint

beijing_starbucks_csv_file = "beijing-starbucks.csv"

header = []
data = []

with open(beijing_starbucks_csv_file, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = next(reader)
    data = list(reader)

pprint(header)
pprint(data[:5])

['id', 'title', 'avgscore', 'areaname', 'backCateName']
[['598894939', '星巴克（北京门头沟长安天街店）', '4.1', '门头沟城区', '咖啡厅'],
 ['195215506', '星巴克（未来中心店）', '4.3', '北七家', '咖啡厅'],
 ['50090428', '星巴克（北京房山北关西路店）', '4.4', '良乡', '咖啡厅'],
 ['192951735', '星巴克（北京昌平悦荟店）', '4.0', '昌平镇', '咖啡厅'],
 ['5823459', '星巴克（北京西站第一店）', '3.9', '北京西站/六里桥', '咖啡厅']]


1. 所有门店按评分从高到低排序

In [2]:
# 把 avgscore 列从 str 类型转换为 float 类型

for row in data:
    row[2] = float(row[2])

pprint(data[:5])

[['598894939', '星巴克（北京门头沟长安天街店）', 4.1, '门头沟城区', '咖啡厅'],
 ['195215506', '星巴克（未来中心店）', 4.3, '北七家', '咖啡厅'],
 ['50090428', '星巴克（北京房山北关西路店）', 4.4, '良乡', '咖啡厅'],
 ['192951735', '星巴克（北京昌平悦荟店）', 4.0, '昌平镇', '咖啡厅'],
 ['5823459', '星巴克（北京西站第一店）', 3.9, '北京西站/六里桥', '咖啡厅']]


In [3]:
# 排序前
print("排序前:")
pprint(data[:5])

# 使用 sorted 函数给 data 按照 avgscore 排序
sorted_data = sorted(data, key=lambda _: _[2], reverse=True)

# 排序后
print("排序后:")
pprint(sorted_data[:5])

排序前:
[['598894939', '星巴克（北京门头沟长安天街店）', 4.1, '门头沟城区', '咖啡厅'],
 ['195215506', '星巴克（未来中心店）', 4.3, '北七家', '咖啡厅'],
 ['50090428', '星巴克（北京房山北关西路店）', 4.4, '良乡', '咖啡厅'],
 ['192951735', '星巴克（北京昌平悦荟店）', 4.0, '昌平镇', '咖啡厅'],
 ['5823459', '星巴克（北京西站第一店）', 3.9, '北京西站/六里桥', '咖啡厅']]
排序后:
[['1186879667', '星巴克（北京大成路物美店）', 4.7, '青塔', '咖啡厅'],
 ['1186879667', '星巴克（北京大成路物美店）', 4.7, '青塔', '咖啡厅'],
 ['1113374444', '星巴克（北京路劲世界城店）', 4.6, '昌平镇', '咖啡厅'],
 ['185739148', '星巴克（北京学院南路店）', 4.6, '北下关', '咖啡厅'],
 ['1457682522', '星巴克（北京橡树澜湾店）', 4.6, '卢沟桥', '咖啡厅']]


2. 通过门店的商圈 （areaname）结合北京区域数据给每个门店添加新字段 area 表示其所属的北京区域（如朝阳区、西城区）

In [4]:
import json

area_data = {}
with open("beijing-area.json", "r", encoding="utf-8") as f:
    area_data = json.load(f)

pprint({k: v[:2] for k, v in area_data.items()})

{'东城区': ['东直门', '安定门'],
 '丰台区': ['洋桥/木樨园', '刘家窑/宋家庄'],
 '大兴区': ['旧宫', '黄村'],
 '密云区': ['司马台长城', '密云县县城'],
 '平谷区': ['平谷城区'],
 '延庆区': ['八达岭', '延庆城区'],
 '怀柔区': ['商业街'],
 '房山区': ['长阳镇', '良乡'],
 '昌平区': ['昌平镇', '回龙观'],
 '朝阳区': ['大望路', '双井'],
 '海淀区': ['双榆树', '上地'],
 '石景山区': ['苹果园', '古城/八角'],
 '西城区': ['西直门/动物园', '复兴门'],
 '通州区': ['梨园', '新华大街'],
 '门头沟区': ['门头沟城区'],
 '顺义区': ['马坡牛栏山镇', '首都机场/机场路']}


In [5]:
# 返回商圈所属于的区
def to_area(areaname: str) -> str:
    for k, v in area_data.items():
        if areaname == k or areaname in v:
            return k
    return "未知"

# 测试
to_area("酒仙桥") == "朝阳区"

True

In [6]:
# 新增 area 列
for row in data:
    row.append(to_area(row[3]))

pprint(data[:5])

[['598894939', '星巴克（北京门头沟长安天街店）', 4.1, '门头沟城区', '咖啡厅', '门头沟区'],
 ['195215506', '星巴克（未来中心店）', 4.3, '北七家', '咖啡厅', '昌平区'],
 ['50090428', '星巴克（北京房山北关西路店）', 4.4, '良乡', '咖啡厅', '房山区'],
 ['192951735', '星巴克（北京昌平悦荟店）', 4.0, '昌平镇', '咖啡厅', '昌平区'],
 ['5823459', '星巴克（北京西站第一店）', 3.9, '北京西站/六里桥', '咖啡厅', '丰台区']]


3. 按照北京区域分组所有门店

In [7]:
grouped_data = {}

# 按照 区 分组
for row in data:
    if row[5] not in grouped_data:
        grouped_data[row[5]] = []
    grouped_data[row[5]].append(row)

pprint({k: v[:1] for k, v in grouped_data.items()})

{'东城区': [['159988078', '星巴克（北京东直门来福士一店）', 4.4, '东直门', '咖啡厅', '东城区']],
 '丰台区': [['5823459', '星巴克（北京西站第一店）', 3.9, '北京西站/六里桥', '咖啡厅', '丰台区']],
 '大兴区': [['50168849', '星巴克臻选（北京欣宁大街店）', 4.2, '西红门', '咖啡厅', '大兴区']],
 '密云区': [['1132796034', '星巴克（北京密云鼓楼南大街店）', 3.8, '密云县县城', '咖啡厅', '密云区']],
 '平谷区': [['760147141', '星巴克（北京平谷国泰店）', 4.3, '平谷城区', '咖啡厅', '平谷区']],
 '延庆区': [['1216288205', '星巴克（北京延庆万达店）', 4.3, '延庆城区', '咖啡厅', '延庆区']],
 '怀柔区': [['175350734', '星巴克（北京怀柔万达店）', 4.0, '商业街', '咖啡厅', '怀柔区']],
 '房山区': [['50090428', '星巴克（北京房山北关西路店）', 4.4, '良乡', '咖啡厅', '房山区']],
 '昌平区': [['195215506', '星巴克（未来中心店）', 4.3, '北七家', '咖啡厅', '昌平区']],
 '朝阳区': [['1564543910', '星巴克（北京三里屯南区NOW店）', 4.3, '三里屯/工体', '咖啡厅', '朝阳区']],
 '未知': [['1457682522', '星巴克（北京橡树澜湾店）', 4.6, '卢沟桥', '咖啡厅', '未知']],
 '海淀区': [['628226657', '星巴克（北京中关村环保园店）', 4.3, '海淀区', '咖啡厅', '海淀区']],
 '石景山区': [['4218587', '星巴克（北京石景山万达广场店）', 4.0, '鲁谷', '咖啡厅', '石景山区']],
 '西城区': [['314057', '星巴克臻选（北京前门大街店）', 4.3, '前门/大栅栏', '咖啡厅', '西城区']],
 '通州区': [['50233524', '星巴克（北京通州万达广场

4. 统计各个区域的门店数量，找出拥有星巴克门店数量最多的三个区域

In [8]:
counter = {}

for area, rows in grouped_data.items():
    counter[area] = len(rows)

print("各个区域门店数量:")
pprint(counter)

# 按门店数量排序
counter = {k:v for k, v in sorted(counter.items(), key=lambda _:_[1], reverse=True)}

# Top 3
print("拥有星巴克门店数量最多的三个区域:")
for k in list(counter.keys())[:3]:
    print(f"{k}: {counter[k]}")

各个区域门店数量:
{'东城区': 37,
 '丰台区': 52,
 '大兴区': 19,
 '密云区': 3,
 '平谷区': 1,
 '延庆区': 3,
 '怀柔区': 2,
 '房山区': 7,
 '昌平区': 16,
 '朝阳区': 210,
 '未知': 9,
 '海淀区': 81,
 '石景山区': 13,
 '西城区': 38,
 '通州区': 24,
 '门头沟区': 3,
 '顺义区': 12}
拥有星巴克门店数量最多的三个区域:
朝阳区: 210
海淀区: 81
丰台区: 52


5. 统计各个区域星巴克门店的平均评分，找出平均评分最好的北京区域

In [9]:
import statistics

area_score = {}
for area, rows in grouped_data.items():
    # 使用 mean 函数计算平均数
    score = statistics.mean([_[2] for _ in rows])
    # 保留两位小数
    area_score[area] = round(score, 2)

print("各个区域星巴克门店的平均评分:")
pprint(area_score)

# 按评分排序
area_score = {k:v for k, v in sorted(area_score.items(), key=lambda _:_[1], reverse=True)}

best_area = list(area_score.items())[0]
print(f"平均评分最好的北京区域: {best_area[0]} {best_area[1]}")

各个区域星巴克门店的平均评分:
{'东城区': 4.12,
 '丰台区': 4.16,
 '大兴区': 4.24,
 '密云区': 4.13,
 '平谷区': 4.3,
 '延庆区': 2.87,
 '怀柔区': 4.15,
 '房山区': 4.39,
 '昌平区': 3.92,
 '朝阳区': 4.11,
 '未知': 2.44,
 '海淀区': 4.08,
 '石景山区': 3.82,
 '西城区': 4.0,
 '通州区': 4.2,
 '门头沟区': 2.67,
 '顺义区': 4.31}
平均评分最好的北京区域: 房山区 4.39


6. 分别找出拥有最多星巴克门店和平均评分最高的商圈（areaname）

In [10]:
# 按商圈分组
areaname_grouped_data = {}
for row in data:
    areaname = row[3]
    if areaname not in areaname_grouped_data:
        areaname_grouped_data[areaname] = []
    areaname_grouped_data[areaname].append(row)

In [11]:
# 计算各个商圈的门店数量和平均评分
areaname_info = []
for areaname, rows in areaname_grouped_data.items():
    score = statistics.mean([_[2] for _ in rows])
    score = round(score, 2)
    areaname_info.append((areaname, len(rows), score))

print("拥有最多星巴克门店的商圈:")
for item in sorted(areaname_info, key=lambda _:_[1], reverse=True)[:5]:
    print(f"{item[0]} : {item[1]} ⭐{item[2]}")

print()

print("拥有最高平均评分的商圈:")
for item in sorted(areaname_info, key=lambda _:_[2], reverse=True)[:5]:
    print(f"{item[0]} : {item[1]} ⭐{item[2]}")

print()

print("拥有最多星巴克门店且平均评分大于 4.1 的商圈")
for item in list(filter(lambda _:_[2] >= 4.1, sorted(areaname_info, key=lambda _:_[1], reverse=True)))[:5]:
    print(f"{item[0]} : {item[1]} ⭐{item[2]}")


拥有最多星巴克门店的商圈:
望京 : 23 ⭐4.26
国贸/建外 : 22 ⭐4.27
大望路 : 17 ⭐3.95
首都机场 : 16 ⭐3.68
朝外大街 : 14 ⭐4.28

拥有最高平均评分的商圈:
北下关 : 1 ⭐4.6
卢沟桥 : 1 ⭐4.6
北大地/万丰路 : 1 ⭐4.6
青塔 : 3 ⭐4.57
八里桥 : 1 ⭐4.5

拥有最多星巴克门店且平均评分大于 4.1 的商圈
望京 : 23 ⭐4.26
国贸/建外 : 22 ⭐4.27
朝外大街 : 14 ⭐4.28
亮马桥/三元桥 : 13 ⭐4.3
亦庄 : 13 ⭐4.18
